# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [55]:
import pandas as pd

In [1]:
ls

CONTRIBUTING.md       README.md             index.ipynb
LICENSE.md            homepage_actions.csv*


In [28]:
df = pd.read_csv("homepage_actions.csv")

In [33]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [37]:
df['action'].value_counts(normalize = True)

view     0.772838
click    0.227162
Name: action, dtype: float64

In [39]:
# how many viewers also clicked
viewer_id = df[df['action'] == 'view']['id'].unique()
clicker_id = df[df['action'] == 'click']['id'].unique()

In [51]:
%%timeit
counter = 0
for x in viewer_id:
    if x in clicker_id:
        counter += 1
counter

23.9 ms ± 1.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
counter = 0
for x in viewer_id:
    if x in clicker_id:
        counter += 1
counter

1860

In [50]:
#set(viewer_id)

In [43]:
viewer_set = set(viewer_id)
clicker_set = set(clicker_id)

In [45]:
len(viewer_set.intersection(clicker_set))

1860

In [49]:
%%timeit
len(viewer_set.intersection(clicker_set))

126 µs ± 5.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [53]:
# clicked but didn't view
clicker_set - viewer_set

set()

In [56]:
# Is there any overlap between the control and experiment groups? 
# so, how do you plan to account for this in your experimental design?

In [58]:
experiment_group = set(df[df['group'] == 'experiment']['id'].unique())
control_group = set(df[df['group'] == 'control']['id'].unique())

experiment_group.intersection(control_group)

set()

In [ ]:
#Your code here

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [59]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [61]:
experiment_group = df[df['group'] == 'experiment'].copy()
experiment_group['dummy'] = 1
experiment = experiment_group.pivot(index = 'id', columns = 'action', values = 'dummy')
experiment.fillna(0, inplace = True)

In [62]:
experiment.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [65]:
control_group = df[df['group'] == 'control'].copy()
control_group['dummy'] = 1
control = control_group.pivot(index = 'id', columns = 'action', values = 'dummy')
control.fillna(0, inplace = True)
control

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0
...,...,...
936786,0.0,1.0
937003,0.0,1.0
937073,0.0,1.0


In [68]:
len(experiment), len(control)

(2996, 3332)

In [69]:
experiment['click'].mean(), control['click'].mean()

(0.3097463284379172, 0.2797118847539016)

In [70]:
from scipy.stats import ttest_ind

In [72]:
ttest_ind(control['click'], experiment['click'], equal_var = False)
# Results are statistically significant 

Ttest_indResult(statistic=-2.615440020788211, pvalue=0.008932805628674201)

In [ ]:
#Your code here

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [ ]:
#Your code here

In [73]:
expected_number_of_clicks = len(experiment) * control['click'].mean()
expected_number_of_clicks

838.0168067226891

In [74]:
experiment['click'].sum()

928.0

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [ ]:
#Your code here

In [76]:
import numpy as np

In [79]:
var = len(experiment) * control['click'].mean() * (1 - control['click'].mean())
std = np.sqrt(var)
std                 

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here

In [81]:
# Z Score = (x − x̅ )/σ


In [80]:
z = (experiment['click'].sum() - expected_number_of_clicks) / std
z

3.6625360854823588

In [82]:
from scipy.stats import norm

In [84]:
norm.sf(z)
# Shows that this is statistically significant

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes, the results roughly match the previous statistical test. Both P-values are below the threshold.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.